<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import os
from sklearn.decomposition import PCA
import pandas as pd


In [ ]:
oly_df = pd.read_csv('data/athlete_events.csv', index_col='ID')
columns=['Name', 'Sex', 'Age','Height', 'Weight', 'Season', 'Sport', 'Event']
oly_df = oly_df[columns + ['Medal']]
oly_df = oly_df.dropna(subset=columns)
oly_df.to_csv('data/athlete_events_fil.csv')
print(oly_df.shape)
oly_df.head()


In [ ]:
pd.Series([1,2,3]).astype(str)[0]

In [ ]:
all_sport_groupdf = oly_df.groupby('Sport').agg(['count', 'std', 'mean'])
all_sport_groupdf.columns = [i[0].lower() + '_' +  i[1] for i in all_sport_groupdf.columns]
all_sport_groupdf = all_sport_groupdf.drop(['height_count', 'weight_count'], axis=1).rename({'age_count' : 'count'}, axis=1)
all_sport_groupdf.head()

In [ ]:
winner_sport_groupdf = oly_df.loc[oly_df['Medal'] == oly_df['Medal'], :].groupby('Sport').agg(['count', 'std', 'mean', 'max', 'min'])
winner_sport_groupdf.columns = [i[0].lower() + '_' +  i[1] for i in winner_sport_groupdf.columns]
winner_sport_groupdf = winner_sport_groupdf.drop(['height_count', 'weight_count'], axis=1).rename({'age_count' : 'count'}, axis=1)

winner_sport_groupdf.head()

In [ ]:
age_value = 30

winner_sport_groupdf.loc[:, 'age_z_score'] = (age_value - winner_sport_groupdf['age_mean']) / winner_sport_groupdf['age_std']

winner_sport_groupdf.head()

In [ ]:
0 % 2

In [ ]:
pca = PCA(n_components=2)

### Do PCA on stats to get X and Y of scatter
winner_sport_groupdf.loc[:, 'PCA_1'] = pca.fit_transform(winner_sport_groupdf.drop('count', axis=1))[:, 0]
winner_sport_groupdf.loc[:, 'PCA_2'] = pca.fit_transform(winner_sport_groupdf.drop('count', axis=1))[:, 1]
winner_sport_groupdf.to_csv("data/oly_winners.csv")

winner_sport_groupdf.head()


In [ ]:
winner_sport_groupdf = oly_df.loc[oly_df['Medal'] == oly_df['Medal'], :].groupby('Sport').agg(['count', 'std', 'mean', 'max', 'min'])
winner_sport_groupdf.columns = [i[0].lower() + '_' +  i[1] for i in winner_sport_groupdf.columns]
winner_sport_groupdf = winner_sport_groupdf.drop(['height_count', 'weight_count'], axis=1).rename({'age_count' : 'count'}, axis=1)
winner_sport_groupdf = winner_sport_groupdf.dropna()

winner_sport_groupdf.head()



In [7]:
os.listdir('./data/athlete_events.csv')

NotADirectoryError: [Errno 20] Not a directory: './data/athlete_events.csv'

In [8]:
def _create_data():
    oly_df = pd.read_csv('./data/athlete_events.csv', index_col='ID')
    columns=['Name', 'Sex', 'Age','Height', 'Weight', 'Season', 'Sport', 'Event']
    oly_df = oly_df[columns + ['Medal']]
    oly_df = oly_df.dropna(subset=columns)

    oly_df['Weight_met'] = oly_df['Weight']
    oly_df['Height_met'] = oly_df['Height']

    oly_df['Weight_emp'] = oly_df['Weight_met'] * 2.2046226218
    oly_df['Height_emp'] = oly_df['Height_met'] * 0.393701


    all_sport_groupdf = oly_df.groupby('Sport').agg(['count', 'std', 'mean'])
    winner_sport_groupdf = oly_df[oly_df['Medal'].notna()].groupby('Sport').agg(['count', 'mean', 'std'])

    all_sport_groupdf = oly_df.groupby('Sport').agg(['count', 'std', 'mean'])
    all_sport_groupdf.columns = [i[0].lower() + '_all_' +  i[1] for i in all_sport_groupdf.columns]
    all_sport_groupdf = all_sport_groupdf.drop(['height_met_all_count', 'height_emp_all_count', 'weight_emp_all_count', 'weight_met_all_count'], axis=1).rename({'age_all_count' : 'all_count'}, axis=1)


    winner_sport_groupdf = oly_df[oly_df['Medal'].notna()].groupby('Sport').agg(['count', 'mean', 'std'])
    winner_sport_groupdf.columns = [i[0].lower() + '_winner_' +  i[1] for i in winner_sport_groupdf.columns]

    sport_df = pd.merge(all_sport_groupdf, winner_sport_groupdf, left_index=True, right_index=True)
    sport_df = sport_df.drop(['height_met_winner_count', 'height_emp_winner_count', 'weight_emp_winner_count', 'weight_met_winner_count'], axis=1).rename({'age_winner_count' : 'winner_count'}, axis=1)
    sport_df = sport_df[sport_df['all_count'] > 50]
    sport_df = sport_df.replace(np.nan, 0)
    oly_df = oly_df.replace(np.nan, 0)#.reset_index()

    total_medals = np.sum(sport_df['winner_count'])
    sport_df['pct_athletes_winners'] = sport_df['winner_count'] / sport_df['all_count']
    sport_df['pct_total_medals'] = sport_df['winner_count'] / total_medals
    sport_df['weight_winner_z_score'] = 0
    sport_df['age_winner_z_score'] = 0
    sport_df['height_winner_z_score'] = 0

    sport_df.loc['Average Olympian', [f'height_emp_winner_mean', f'weight_emp_winner_mean']] = [oly_df[f'Height_emp'].mean(), oly_df[f'Weight_emp'].mean()]
    sport_df.loc['Average Olympian', [f'height_met_winner_mean', f'weight_met_winner_mean']] = [oly_df[f'Height_met'].mean(), oly_df[f'Weight_met'].mean()]
    sport_df.loc['Average Olympian', 'age_winner_mean'] = oly_df[f'Age'].mean()

    sport_df.loc['Average Olympian', 'pct_athletes_winners'] = 2
    sport_df.loc['Person', :] = 0
    sport_df.loc['Person', 'pct_athletes_winners'] = 2
    sport_df.to_csv('sport_df_proc.csv')
    oly_df.to_csv('oly_df_proc.csv')
    return sport_df, oly_df

_create_data()

(                           all_count  age_all_std  age_all_mean  \
 Sport                                                             
 Alpine Skiing                 6322.0     3.944300     23.338184   
 Archery                       1883.0     6.712389     26.272969   
 Athletics                    32374.0     4.407144     25.330203   
 Badminton                     1362.0     3.883089     25.800294   
 Baseball                       846.0     4.550422     26.309693   
 Basketball                    3668.0     3.906572     25.453926   
 Beach Volleyball               538.0     4.198318     29.135688   
 Biathlon                      4615.0     4.241076     26.657855   
 Bobsleigh                     2205.0     4.922222     28.650794   
 Boxing                        4363.0     3.411161     23.286729   
 Canoeing                      5550.0     4.478489     25.410991   
 Cross Country Skiing          7529.0     4.270676     26.088724   
 Curling                        410.0     6.8014